Step 1: Load the data and prepare the input data for Meridian

In [2]:
from meridian.data import data_frame_input_data_builder
import pandas as pd
import warnings # Suppress warnings from Meridian about missing data

warnings.filterwarnings('ignore')

mm = pd.read_csv('/Users/thomas.goldvarg/PycharmProjects/funnel-insights-lab/marketing_mix_model/data/mmm_daily_agg.csv', parse_dates=['click_date'])

# different column names in sql query
column_mapping = {
    'clicks_google': 'clicks_Google',
    'clicks_meta': 'clicks_Meta',
    'clicks_linkedin': 'clicks_LinkedIn',
    'spend_google': 'spend_Google',
    'spend_meta': 'spend_Meta',
    'spend_linkedin': 'spend_LinkedIn',
    'click_date': 'time'  # Rename time column as well
}

mm = mm.rename(columns=column_mapping)

# Create a Meridian data frame input data builder
df_builder = data_frame_input_data_builder.DataFrameInputDataBuilder(
    kpi_type='revenue',
    default_kpi_column='revenue_total'
)

# Provide KPI
builder = df_builder.with_kpi(mm, time_col='time')

# Add controls
control_columns = ['holiday_flag', 'promo_discount', 'competitor_index']
builder = builder.with_controls(mm, control_cols=control_columns, time_col='time')

# Define media variables
channels = ['Google','Meta','LinkedIn']
media_columns       = [f'clicks_{ch}' for ch in channels]
media_spend_columns = [f'spend_{ch}'  for ch in channels]

builder = builder.with_media(
    mm,
    media_cols=media_columns,
    media_spend_cols=media_spend_columns,
    media_channels=channels
)

# Build the input_data object
input_data = builder.build()


Step 2: Specify the model configuration

In [3]:
import tensorflow_probability as tfp
import tensorflow as tf
from meridian.model import prior_distribution, spec, model

tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')

# ROI priors based on Meridian documentation: https://developers.google.com/meridian/notebook/meridian-getting-started#:~:text=to%20calibrate%20the%20model%20directly,9

roi_mu, roi_sigma = 0.2, 0.9
priors = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name='roi_m')
)

# Choose model settings
model_spec = spec.ModelSpec(
    prior=priors, # prior distribution for model params. In this case we use normal distribution for ROI
    max_lag=14, # 14 day window effect of channels
    hill_before_adstock=False, # controls order of transformations applied to media variables
    knots=10 # number of control points for modeling baseline
)

mmm = model.Meridian(input_data=input_data, model_spec=model_spec)

print("Starting model training with surpressed TF warnings")


Starting model training with surpressed TF warnings


I0000 00:00:1754242332.989020  333591 service.cc:148] XLA service 0x6000011c8b00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754242332.989110  333591 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1754242333.008741  333591 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Step 3: Sample the model

In [4]:
# Prior sampling - your setting is fine
mmm.sample_prior(300)

# More robust posterior sampling for daily MMM
mmm.sample_posterior(
    n_chains=4,
    n_adapt=2000,    # More adaptation for complex daily patterns
    n_burnin=1000,   # More burn-in to ensure convergence
    n_keep=1500,     # More samples for stable estimates
    seed=0
)

2025-08-03 19:32:42.316097: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1754242363.291298  333591 assert_op.cc:38] Ignoring Assert operator mcmc_retry_init/assert_equal_1/Assert/AssertGuard/Assert


Step 4: Model diagnostics

4.1 Convergence diagnostics

In [5]:
from meridian.analysis import visualizer

# Create analyzer object
diagnostics = visualizer.ModelDiagnostics(mmm)

# Convergence diagnostics
diagnostics.plot_rhat_boxplot()


alt.LayerChart(...)

We ran an r-hat diagnostic to check convergence of the model. The r-hat value should be close to 1.0 for all parameters, indicating that the chains have converged. A common rule of thumb is < 1.10 for acceptable results. Our results are all extremely close to 1.0 which is to be expected considering that we used perfect dummy-data to train the model. This is an indicator that the model has learned the underlying patterns in the data effectively and we can make decisions based on it.

4.2 Model Fit

In [6]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

alt.LayerChart(...)

The model replicates the overall trend in the data, capturing the peaks and troughs in revenue. The model fit is good, indicating that the model has learned the underlying patterns in the data effectively.

Step 5: Analyze the results

5.1 Channel Contribution Analysis

In [7]:
media_summary = visualizer.MediaSummary(mmm)

# Contribution of media channels
media_summary.plot_channel_contribution_area_chart()

alt.Chart(...)

In [8]:
media_summary.plot_channel_contribution_bump_chart()

alt.Chart(...)

In [9]:
media_summary.plot_contribution_waterfall_chart()

alt.LayerChart(...)

In [10]:
media_summary.plot_contribution_pie_chart()

alt.LayerChart(...)

5.2 Spend vs Contribution to ROI

In [11]:
media_summary.plot_spend_vs_contribution()

alt.FacetChart(...)

In [12]:
media_summary.plot_roi_bar_chart()

alt.LayerChart(...)

5.3 Response and Saturation Curves

These curves show the relationship between media spend and the response (revenue) generated by each channel. The response curve shows how much revenue is generated for each dollar spent on media, while the saturation curve shows how much additional revenue is generated as spend increases. The curves can help identify the optimal level of spend for each channel to maximize ROI.

In [13]:
media_effects = visualizer.MediaEffects(mmm)

media_effects.plot_response_curves()

alt.FacetChart(...)

In [14]:
media_effects.plot_adstock_decay()

alt.FacetChart(...)

In [15]:
hill_chart = media_effects.plot_hill_curves()
hill_chart['media']

alt.FacetChart(...)

Step 6: Create a Summary Report

In [ ]:
from meridian.analysis import summarizer

summary = summarizer.Summarizer(mmm)

summary.output_model_results_summary(
    filename='mmm_summary.html',
    filepath='//marketing_mix_model/results/mmm_summary.html',
)